<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/03_01_char_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tensorflow 2.x와 관련 패키지 임포트

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils  import to_categorical

### 트레이닝 시퀀스(training sequence) 정의

In [0]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)
print ("Length of 'test sentence' is %s" %len(sentence))

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 180


### 문자와 사전 정의

In [0]:
char_set = list(set(sentence))
mapping = {w: i for i, w in enumerate(char_set)}
print ("CHARACTERS: ")
print (len(char_set))
print (char_set)
print ("DICTIONARY: ")
print (len(mapping))
print (mapping)

CHARACTERS: 
25
['t', 'h', "'", 'c', ',', 'k', 'y', 'b', 'p', 'o', 's', 'f', 'l', 'a', 'd', 'g', 'n', 'u', ' ', 'w', 'e', 'i', 'r', 'm', '.']
DICTIONARY: 
25
{'t': 0, 'h': 1, "'": 2, 'c': 3, ',': 4, 'k': 5, 'y': 6, 'b': 7, 'p': 8, 'o': 9, 's': 10, 'f': 11, 'l': 12, 'a': 13, 'd': 14, 'g': 15, 'n': 16, 'u': 17, ' ': 18, 'w': 19, 'e': 20, 'i': 21, 'r': 22, 'm': 23, '.': 24}


### 네트워크 설정

In [0]:
data_dim    = len(char_set)
num_classes = len(char_set)
hidden_size     = 64
sequence_length = 10  # any arbitrary number
print ('data_dim : %d' %data_dim)
print ('num_classes : %d' %num_classes)

data_dim : 25
num_classes : 25


### 훈련용 시퀀스 생성

In [0]:
def print_list(_name,_x):
    print('Type of [%s] is [%s]' %(_name,type(_x)))
    print('Length of [%s] is %s' %(_name,len(_x)))
    print('%s[0] Looks like %s' %(_name,_x[0]))

In [0]:
dataX = [] # 170 sequences
dataY = [] # 170 sequences
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x = [mapping[c] for c in x_str]  # x str to index
    y = [mapping[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)
    if i<5:
        print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*10, x, y))

print('\n')
print ((type(dataX)))
print (dataX[0])
print (dataX[1])
print (dataX[168])
print (dataX[169])
#print_list('dataX',dataX)

print('\n')
print ((type(dataY)))
print (dataY[0])
print (dataY[1])
print (dataY[168])
print (dataY[169])
#print_list('dataY',dataY)

[  0/180] [if you wan]=>[f you want]
          [21, 11, 18, 6, 9, 17, 18, 19, 13, 16]=>[11, 18, 6, 9, 17, 18, 19, 13, 16, 0]
[  1/180] [f you want]=>[ you want ]
          [11, 18, 6, 9, 17, 18, 19, 13, 16, 0]=>[18, 6, 9, 17, 18, 19, 13, 16, 0, 18]
[  2/180] [ you want ]=>[you want t]
          [18, 6, 9, 17, 18, 19, 13, 16, 0, 18]=>[6, 9, 17, 18, 19, 13, 16, 0, 18, 0]
[  3/180] [you want t]=>[ou want to]
          [6, 9, 17, 18, 19, 13, 16, 0, 18, 0]=>[9, 17, 18, 19, 13, 16, 0, 18, 0, 9]
[  4/180] [ou want to]=>[u want to ]
          [9, 17, 18, 19, 13, 16, 0, 18, 0, 9]=>[17, 18, 19, 13, 16, 0, 18, 0, 9, 18]


<class 'list'>
[21, 11, 18, 6, 9, 17, 18, 19, 13, 16]
[11, 18, 6, 9, 17, 18, 19, 13, 16, 0]
[18, 9, 11, 18, 0, 1, 20, 18, 10, 20]
[9, 11, 18, 0, 1, 20, 18, 10, 20, 13]


<class 'list'>
[11, 18, 6, 9, 17, 18, 19, 13, 16, 0]
[18, 6, 9, 17, 18, 19, 13, 16, 0, 18]
[9, 11, 18, 0, 1, 20, 18, 10, 20, 13]
[11, 18, 0, 1, 20, 18, 10, 20, 13, 24]


### 훈련용 시퀀스 데이타 리쉐이프(reshape)

In [0]:
# separate into input and output
dataX = np.array(dataX)
print (dataX.shape)

dataY = np.array(dataY)
print (dataY.shape)

X = dataX[:,:]
y = dataY[:,-1]
print (X.shape) # (170, 10)
print (y.shape) # (170,)

(170, 10)
(170, 10)
(170, 10)
(170,)


In [0]:
sequences = [to_categorical(x, num_classes = data_dim) for x in X]
print (sequences[0].shape)   # (10,25)
print (sequences[169].shape) # (10,25)

X = np.array(sequences)
print (X.shape) # (170, 10, 25)

print (y.shape) # (170,)
y = to_categorical(y, num_classes = data_dim)
print (y.shape) # (170, 25)
print (y[0])  # t -> 4th one-hot

(10, 25)
(10, 25)
(170, 10, 25)
(170,)
(170, 25)
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


### 모델 설계

In [0]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(sequence_length, num_classes)))
model.add(Dense(data_dim, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 75)                30300     
_________________________________________________________________
dense (Dense)                (None, 25)                1900      
Total params: 32,200
Trainable params: 32,200
Non-trainable params: 0
_________________________________________________________________
None


### 훈련 셋업 및 훈련

In [0]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
print (X.shape) # (170, 10, 25)
print (y.shape) # (170, 25)
model.fit(X, y, epochs=120, verbose=2)

(170, 10, 25)
(170, 25)
Train on 170 samples
Epoch 1/120
170/170 - 5s - loss: 3.2120 - accuracy: 0.0471
Epoch 2/120
170/170 - 0s - loss: 3.1756 - accuracy: 0.1412
Epoch 3/120
170/170 - 0s - loss: 3.1313 - accuracy: 0.1882
Epoch 4/120
170/170 - 0s - loss: 3.0585 - accuracy: 0.1882
Epoch 5/120
170/170 - 0s - loss: 2.9425 - accuracy: 0.1882
Epoch 6/120
170/170 - 0s - loss: 2.8784 - accuracy: 0.1882
Epoch 7/120
170/170 - 0s - loss: 2.8480 - accuracy: 0.1882
Epoch 8/120
170/170 - 0s - loss: 2.8229 - accuracy: 0.1882
Epoch 9/120
170/170 - 0s - loss: 2.8161 - accuracy: 0.1882
Epoch 10/120
170/170 - 0s - loss: 2.8067 - accuracy: 0.1882
Epoch 11/120
170/170 - 0s - loss: 2.7998 - accuracy: 0.1882
Epoch 12/120
170/170 - 0s - loss: 2.7856 - accuracy: 0.1882
Epoch 13/120
170/170 - 0s - loss: 2.7745 - accuracy: 0.1882
Epoch 14/120
170/170 - 0s - loss: 2.7670 - accuracy: 0.1882
Epoch 15/120
170/170 - 0s - loss: 2.7637 - accuracy: 0.1882
Epoch 16/120
170/170 - 0s - loss: 2.7615 - accuracy: 0.1882
Epoc

## 훈련후 텍스트 생성 체크 목적

In [0]:
from pickle import load
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		#encoded = encoded.reshape(1, encoded.shape[0], encoded.shape[1])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

Sentence : if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of 'test sentence' is 180

In [0]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'want to bu', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'collect wo', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'rather tea', 20))

want to build a ship, don't dr
collect wood and don't assign 
rather teach them to lo g fo d
